In [1]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
import random
import time

In [3]:
# simplified form from Appendix A of 2410.21260

def evolve_Nc(delta, eta_max, ICs = "ones"):
    nsteps = round(eta_max/delta)
    d2 = delta**2
    
    G = np.ones((nsteps + 1, nsteps + 1))
    G2 = np.ones((nsteps + 1, nsteps + 1))
    Gm = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm2 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

    I3 = np.ones((nsteps + 1, nsteps + 1))
    I4 = np.ones((nsteps + 1, nsteps + 1))
    I5 = np.ones((nsteps + 1, nsteps + 1))
    It = np.ones((nsteps + 1, nsteps + 1))
    Gm3 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm4 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm5 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    
    if type(ICs) != str:
        for amp, ne_amp, ic in zip([G, G2, I3, I4, I5], [Gm, Gm2, Gm3, Gm4, Gm5], ICs):
            amp *= ic
            ne_amp *= ic
        It *= ICs[5]
        
    print("amplitudes initialized to", ICs)
        
    for j in range(1, nsteps + 1):
        for i in range(j):
            
            G[i, j] = G[i, j-1]
            G[i, j] += d2*np.sum(Gm[i, i:j, j-1] + 3*G[i:j, j-1] + 2*G2[i:j, j-1] + 2*Gm2[i, i:j, j-1])
            
            Gm[i, i, j] = G[i, j]
            for k in range(i+1, j+1):
                Gm[i, k, j] = Gm[i, k-1, j-1]
                Gm[i, k, j] += d2*np.sum(Gm[i, k-1:j, j-1] + 3*G[k-1:j, j-1] + 2*G2[k-1:j, j-1] + 2*Gm2[i, k-1:j, j-1])

            G2[i, j] = G2[i, j-1]
            G2[i, j] += 2*d2*np.trace(G[:i, j-i:j] + 2*G2[:i, j-i:j])

            Gm2[i, i, j] = G2[i, j]
            Gm2[i, i+1:j+1, j] = Gm2[i, i:j, j-1]

            I3[i, j] = I3[i, j-1]
            I3[i, j] += 0.5*d2*np.sum(2*Gm3[i, i:j, j-1] - 4*Gm4[i, i:j, j-1] + 2*Gm5[i, i:j, j-1] - 2*Gm2[i, i:j, j-1])
            I3[i, j] += 0.5*d2*np.trace(4*I3[:i, j-i:j] - 4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 4*G[:i, j-i:j] - 6*G2[:i, j-i:j])

            Gm3[i, i, j] = I3[i, j]
            for k in range(i+1, j+1):
                Gm3[i, k, j] = Gm3[i, k-1, j-1] 
                Gm3[i, k, j] += 0.5*d2*np.sum(2*Gm3[i, k-1:j, j-1] - 4*Gm4[i, k-1:j, j-1] + 2*Gm5[i, k-1:j, j-1] - 2*Gm2[i, k-1:j, j-1])
            
            I4[i, j] = I4[i, j-1]
            I4[i, j] += 0.5*d2*np.trace(4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 2*G[:i, j-i:j] - 3*G2[:i, j-i:j])
            
            Gm4[i, i, j] = I4[i, j]
            Gm4[i, i+1:j+1, j] = Gm4[i, i:j, j-1]
            
            I5[i, j] = I5[i, j-1]
            I5[i, j] += 0.5*d2*np.trace(-2*I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j] + 4*G[:i, j-i:j] + 7*G2[:i, j-i:j])
            
            Gm5[i, i, j] = I5[i, j]
            Gm5[i, i+1:j+1, j] = Gm5[i, i:j, j-1]

            It[i, j] = It[i, j-1]
            It[i, j] -= 0.5*d2*np.trace(G[:i, j-i:j] + 3*G2[:i, j-i:j] - I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j])
                
        if(np.mod(j, round(nsteps/10)) == 0): 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")

    return [G, G2, I3, I4, I5, It]
    

In [75]:
# large Nc 
eta_maxs = {0.0125:10, 0.016:10, 0.025:20, 0.032:20, 0.0375:30, 0.05:40, 0.0625:50, 0.075:60, 0.08:60, 0.1:70}

#     ICs = np.random.randint(-6,6, size=(6))
ICs = "ones"
ICs_tag = "ones"
output_dir = "/Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc/"

for delta, eta_max in eta_maxs.items():
    # if delta != 0.05: continue 
    # eta_max = 4 # testing
    eta_max = 300*delta # limit to 300 steps for fast data

    if round(eta_max/delta) > 800:
        print("Error: nsteps > 800 and will probably crash your computer") 
        break
        
    print("nsteps:", round(eta_max/delta)) 
    delta_str = str(delta)[2:]
    amp_strs = {0:"G", 1:"G2", 2:"I3", 3:"I4", 4:"I5", 5:"It"}
    print("Staring (d, eta_max)=", delta, eta_max)
    
    output = evolve_Nc(delta, eta_max, ICs = ICs)

    # save output
    if not os.path.exists(output_dir+"d"+delta_str+"_"+ICs_tag): os.mkdir(output_dir+"d"+delta_str+"_"+ICs_tag)
    for iamp in range(6):
        np.savetxt(output_dir+"d"+delta_str+"_"+ICs_tag+"/d"+delta_str+"_Nc_"+ICs_tag+"_"+amp_strs[iamp]+".dat", output[iamp])
    print("wrote out amplitudes to", output_dir)
    output = 0
    

nsteps: 300
Staring (d, eta_max)= 0.0125 3.75
amplitudes initialized to ones
10.0% done (17:23.02)
20.0% done (17:23.02)
30.0% done (17:23.03)
40.0% done (17:23.06)
50.0% done (17:23.10)
60.0% done (17:23.17)
70.0% done (17:23.26)
80.0% done (17:23.39)
90.0% done (17:23.55)
100.0% done (17:24.16)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc/
nsteps: 300
Staring (d, eta_max)= 0.016 4.8
amplitudes initialized to ones
10.0% done (17:24.17)
20.0% done (17:24.17)
30.0% done (17:24.19)
40.0% done (17:24.21)
50.0% done (17:24.25)
60.0% done (17:24.32)
70.0% done (17:24.41)
80.0% done (17:24.54)
90.0% done (17:25.10)
100.0% done (17:25.31)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc/
nsteps: 300
Staring (d, eta_max)= 0.025 7.5
amplitudes initialized to ones
10.0% done (17:25.31)
20.0% done (17:25.32)
30.0% done (17:25.33)
40.0% done (17:25.36)
50.0% done (17:25.40)
60.0% done (17:25.47)
7

In [5]:
# here lambda is a cutoff, also includes Qt corrections from 2406.11647 and It amplitude from 2410.21260

def evolve_NcNf(delta, eta_max, Nf=3.0, ICs="ones"):
    Nr = 0.25*(Nf/3.0)
    nsteps = round(eta_max/delta)
    d2 = delta**2
    
    G = np.ones((nsteps + 1, nsteps + 1))
    Q = np.ones((nsteps + 1, nsteps + 1))
    G2 = np.ones((nsteps + 1, nsteps + 1))
    Qt = np.ones((nsteps + 1, nsteps + 1))
    GmT = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    GmB = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm2 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

    I3t = np.ones((nsteps + 1, nsteps + 1))
    I3 = np.ones((nsteps + 1, nsteps + 1))
    I4 = np.ones((nsteps + 1, nsteps + 1))
    I5 = np.ones((nsteps + 1, nsteps + 1))
    It = np.ones((nsteps + 1, nsteps + 1))
    Gm3t = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm3 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm4 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm5 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

            
    if type(ICs) != str:  
        for amp, ne_amp, ic in zip([G, Q, G2, I3t, I3, I4, I5], [GmT, GmB, Gm2, Gm3t, Gm3, Gm4, Gm5], ICs):
            amp *= ic
            ne_amp *= ic
            
    elif "basis" in ICs:
        pass
        # FIXME: add support for basis functions
    
    # basis_func = (a - eta), (b - s_10), (c - const)
    # basis_amp = 0 - 7 (G, Q, G2, I3t, I3, I4, I5, I6)
    
    # amps = [G, Q, G2, I3t, I3, I4, I5, I6]
    # neighs = [GmT, GmB, Gm2, Gm3t, Gm3, Gm4, Gm5, Gm6]
    
    # if basis_func == "c":
    #     amps[basis_amp] += 1
    #     neighs[basis_amp] += 1
        
    # elif basis_func == "b":
    #     amps[basis_amp] +=
    #         if ia != basis_amp: continue
    #         amp += 1
                  
    #     amp = np.array([[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
    #     Gm = np.array([[[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])

     
    print("amplitudes initialized to", ICs)
    print("Nf=", Nf)
        
    for j in range(1, nsteps + 1):
        for i in range(j):
            
            Q[i, j] = Q[i, j-1]
            Q[i, j] += 0.5*d2*np.sum(Q[i:j-1, j-1] + 2*G2[i:j-1, j-1]) 
            Q[i, j] += 0.5*d2*np.trace(Q[:i, j-i:j] + 2*G2[:i, j-i:j]) 
            Q[i, j] += d2*np.sum(2*G[i:j-1,j-1] + Q[i:j-1,j-1] + 2*G2[i:j-1,j-1] + 2*GmT[i, i:j-1, j-1] - GmB[i, i:j-1, j-1] + 2*Gm2[i, i:j-1, j-1])
            
            GmB[i, i, j] = Q[i, j]
            for k in range(i+1, j+1):
                GmB[i, k, j] = GmB[i, k-1, j-1]
                GmB[i, k, j] += 0.5*d2*np.sum(Q[k-1:j-1, j-1] + 2*G2[k-1:j-1, j-1])
                ii = max(i, k-1)
                GmB[i, k, j] += d2*np.sum(2*GmT[i, ii:j-1, j-1] - GmB[i, ii:j-1, j-1] + 2*Gm2[i, ii:j-1, j-1] + 2*G[ii:j-1, j-1] + Q[ii:j-1, j-1] + 2*G2[ii:j-1, j-1])
            
            G[i, j] = G[i, j-1]
            G[i, j] -= Nr*d2*np.trace(Q[:i, j-i:i] + 2*G2[:i, j-i:i])
            G[i, j] += d2*np.sum(3*G[i:j-1, j-1] + 2*G2[i:j-1, j-1] - 2*Nr*Qt[i:j-1, j-1])
            G[i, j] += d2*np.sum(GmT[i, i:j-1, j-1] + (2-2*Nr)*Gm2[i, i:j-1, j-1] - Nr*GmB[i, i:j-1, j-1])
            
            GmT[i, i, j] = G[i, j]
            for k in range(i+1, j+1):
                GmT[i, k, j] = GmT[i, k-1, j-1]
                ii = max(i, k-1)
                GmT[i, k, j] += d2*np.sum(GmT[i, ii:j-1, j-1] + (2-2*Nr)*Gm2[i, ii:j-1, j-1] - Nr*GmB[i, ii:j-1, j-1] + 3*G[ii:j-1, j-1] + 2*G2[ii:j-1, j-1] - 2*Nr*Qt[ii:j-1, j-1])
            
            G2[i, j] = G2[i, j-1]
            G2[i, j] += 2*d2*np.trace(G[:i, j-i:j] + 2*G2[:i, j-i:j])

            Gm2[i, i, j] = G2[i, j]
            Gm2[i, i+1:j+1, j] = Gm2[i, i:j, j-1]

            Qt[i, j] = Qt[i, j-1]
            Qt[i, j] -= d2*np.trace(Q[:i, j-i:j] + 2*G2[:i, j-i:j])
              
            I3[i, j] = I3[i, j-1]
            I3[i, j] += 0.5*d2*np.sum(4*Gm3t[i, i:j-1, j-1] - 2*Gm3[i, i:j-1, j-1] - 4*Gm4[i, i:j-1, j-1] + 2*Gm5[i, i:j-1, j-1] - 2*Gm2[i, i:j-1, j-1])
            I3[i, j] += 0.5*d2*np.trace(4*I3t[:i, j-i:j] - 4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 4*G[:i, j-i:j] - 6*G2[:i, j-i:j])
            
            Gm3[i, i, j] = I3[i, j]
            for k in range(i+1, j+1):
                Gm3[i, k, j] = Gm3[i, k-1, j-1]
                ii = max(i, k-1)
                Gm3[i, k, j] += d2*np.sum(2*Gm3t[i, ii:j-1, j-1] - Gm3[i, ii:j-1, j-1] - 2*Gm4[i, ii:j-1, j-1] + Gm5[i, ii:j-1, j-1] - Gm2[i, ii:j-1, j-1])
            
            I3t[i, j] = I3t[i, j-1]
            I3t[i, j] += 0.5*d2*np.sum(2*Gm3t[i, i:j-1, j-1] - 2*Nr*Gm3[i, i:j-1, j-1] + (4*Nr - 4)*Gm4[i, i:j-1, j-1] + (2 - 2*Nr)*Gm5[i, i:j-1, j-1] + (2*Nr - 2)*Gm2[i, i:j-1, j-1])
            I3t[i, j] += 0.5*d2*np.trace(-2*Nr*I3[:i, j-i:j] + 4*I3t[:i, j-i:j] + (4*Nr-4)*I4[:i, j-i:j] + (2 - 2*Nr)*I5[:i, j-i:j] - 4*G[:i, j-i:j] + (2*Nr - 6)*G2[:i, j-i:j])
            
            Gm3t[i, i, j] = I3t[i, j]
            for k in range(i+1, j+1):
                Gm3t[i, k, j] = Gm3t[i, k-1, j-1]
                Gm3t[i, k, j] += d2*np.sum(Gm3t[i, ii:j-1, j-1] - Nr*Gm3[i, ii:j-1, j-1] + (2*Nr - 2)*Gm4[i, ii:j-1, j-1] + (1 - Nr)*Gm5[i, ii:j-1, j-1] + (Nr - 1)*Gm2[i, ii:j-1, j-1])

            I4[i, j] = I4[i, j-1]
            I4[i, j] += 0.5*d2*np.trace(4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] + G2[:i, j-i:j])
            
            Gm4[i, i, j] = I4[i, j]
            Gm4[i, i+1:j+1, j] = Gm4[i, i:j, j-1]
            
            I5[i, j] = I5[i, j-1]
            I5[i, j] += 0.5*d2*np.trace(-2*I3t[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j] + 2*G[:i, j-i:j] + 3*G2[:i, j-i:j])
            
            Gm5[i, i, j] = I5[i, j]
            Gm5[i, i+1:j+1, j] = Gm5[i, i:j, j-1]

            It[i, j] = It[i, j-1]
            It[i, j] -= 0.5*d2*np.trace(Q[:i, j-i:j] + 3*G2[:i, j-i:j] - I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j])
                
        if(np.mod(j, round(nsteps/10)) == 0): 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")

    return [Q, G, G2, Qt, I3, I3t, I4, I5, It]
    

In [11]:
# large Nc&Nf (can handle 6/9 = .6 of the memory of large Nc)
eta_maxs = {0.0125:6, 0.016:6, 0.025:12, 0.032:12, 0.0375:18, 0.05:24, 0.0625:30, 0.075:36, 0.08:36, 0.1:42}

#     ICs = np.random.randint(-6,6, size=(6))
ICs = "ones"
ICs_tag = "ones"
output_dir = "/Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/"

Nf = 3
count = 0
for delta, eta_max in eta_maxs.items():
    # eta_max = 4 # testing
    # eta_max = 300*delta # limit to 300 steps for fast data
    eta_max = 5
    delta = 0.01
    if count > 1: break
    count += 1

    if round(eta_max/delta) > 800:
        print("Error: nsteps > 800 and will probably crash your computer") 
        break
        
    print("nsteps:", round(eta_max/delta)) 
    delta_str = str(delta)[2:]
    amp_strs = {0:"Q", 1:"G", 2:"G2", 3:"Qt", 4:"I3", 5:"I3t", 6:"I4", 7:"I5", 8:"It"}
    print("Staring (d, eta_max)=", delta, eta_max)
    
    output = evolve_NcNf(delta, eta_max, ICs = ICs)

    # save output
    if not os.path.exists(output_dir+"d"+delta_str+"_"+ICs_tag): os.mkdir(output_dir+"d"+delta_str+"_"+ICs_tag)
    for iamp in range(9):
        np.savetxt(output_dir+"d"+delta_str+"_"+ICs_tag+"/d"+delta_str+"_NcNf"+str(Nf)+"_"+ICs_tag+"_"+amp_strs[iamp]+".dat", output[iamp])
    print("wrote out amplitudes to", output_dir)
    output = 0
    

nsteps: 500
Staring (d, eta_max)= 0.01 5
amplitudes initialized to ones
Nf= 3.0
10.0% done (15:04.32)
20.0% done (15:04.37)
30.0% done (15:04.50)
40.0% done (15:05.16)
50.0% done (15:06.00)
60.0% done (15:07.10)
70.0% done (15:08.50)
80.0% done (15:11.08)
90.0% done (15:14.12)
100.0% done (15:18.10)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/
nsteps: 500
Staring (d, eta_max)= 0.01 5
amplitudes initialized to ones
Nf= 3.0
10.0% done (15:18.12)
20.0% done (15:18.17)
30.0% done (15:18.31)
40.0% done (15:18.59)


KeyboardInterrupt: 

In [9]:
# here, lambda is a scale characterizing the target (not a cutoff)
# also includes Qt corrections from 2406.11647 BUT DOES NOT INCLUDE It amplitude from 2410.21260
# also includes running coupling

def alpha_s(coupling, s):
    Nc = 3.0
    Nf = 3.0
    beta2 = (11*Nc-2*Nf)/(12*np.pi)
    s0 = 1.0 # log ratio of IR cutoff to \Lambda_{QCD}
    
    if coupling == 'fixed': return 0.25
    elif coupling == 'running': return (np.sqrt(Nc/(2*np.pi))/beta2)*(1/(s0+s))

def evolve_NcNf_nocutoff(delta, eta_max, Nf=3.0, IC = "ones", verb=True, coupling='fixed'):
    nsteps = round(eta_max/delta)
    d = delta**2
    cp = coupling
    
    Nc = 3.0
    Nr = 0.25*(Nf/Nc)
    
    if IC == "ones":
        Q = np.ones((nsteps + 1, nsteps + 1))
        G = np.ones((nsteps + 1, nsteps + 1))
        G2 = np.ones((nsteps + 1, nsteps + 1))
        Qt = np.ones((nsteps + 1, nsteps + 1))
        I3 = np.ones((nsteps + 1, nsteps + 1))
        I3t = np.ones((nsteps + 1, nsteps + 1))
        I4 = np.ones((nsteps + 1, nsteps + 1))
        I5 = np.ones((nsteps + 1, nsteps + 1))
        
        Gb = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm2 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm3 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm3t = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm4 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm5 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        
    print('Finished initialization')
    
    for j in range(1, nsteps + 1):
        for i in range(j - nsteps, j):
            
            # if i < 0: continue # testing
            
            li = i-j+nsteps
            pli = i-(j-1)+nsteps
            
            # increment Q
            Q[li, j] = Q[pli, j-1]
            Q[li, j] += d*np.sum(alpha_s(cp,delta*np.arange(pli, nsteps))*(2*Gm[pli, i+nsteps:j-1+nsteps, j-1] + 2*G[pli:nsteps, j-1] + Q[pli:nsteps, j-1] - Gb[pli, i+nsteps:j-1+nsteps, j-1] + 2*Gm2[pli, i+nsteps:j-1+nsteps, j-1] + 2*G2[pli:nsteps, j-1]))
            Q[li, j] += 0.5*d*np.sum(alpha_s(cp,delta*np.arange(li, nsteps))*(Q[li:nsteps, j-1] + 2*G2[li:nsteps, j-1]))
            Q[li, j] += 0.5*d*alpha_s(cp,delta*li)*np.sum(Q[li, :j-1] + 2*G2[li, :j-1])

             # increment Gmb (Gamma bar)
            Gb[li, i+nsteps, j] = Q[li, j]
            for k in range(i+1, j+1):
                Gb[li, k+nsteps, j] = Gb[pli, k-1+nsteps, j-1]
                Gb[li, k+nsteps, j] += 0.5*d*np.sum(alpha_s(cp,delta*np.arange(k-j+nsteps, nsteps))*(Q[k-j+nsteps:nsteps, j-1] + 2*G2[k-j+nsteps:nsteps, j-1]))
                lb = max(i, k-1)
                Gb[li, k+nsteps, j] += d*np.sum(alpha_s(cp,delta*np.arange(lb+nsteps-j+1, nsteps))*(2*Gm[pli, lb+nsteps:j-1+nsteps, j-1] + 2*G[lb+nsteps-j+1:nsteps, j-1] + Q[lb+nsteps-j+1:nsteps, j-1] - Gb[pli, lb+nsteps:j-1+nsteps, j-1] + 2*Gm2[pli, lb+nsteps:j-1+nsteps, j-1] + 2*G2[lb+nsteps-j+1:nsteps, j-1]))
                 
            # increment G 
            G[li, j] = G[pli, j-1] 
            G[li, j] += -d*Nr*alpha_s(cp,delta*li)*np.sum(Q[li, :j] + 2*G2[li, :j])
            G[li, j] += d*np.sum(alpha_s(cp,delta*np.arange(pli, nsteps))*(Gm[pli, i+nsteps:j-1+nsteps, j-1] + 3*G[pli:nsteps, j-1] + 2*G2[pli:nsteps, j-1] + (2 - 2*Nr)*Gm2[pli, i+nsteps:j-1+nsteps, j-1] - Nr*Gb[pli, i+nsteps:j-1+nsteps, j-1] - 2*Nr*Qt[pli:nsteps, j-1]))
                
            # increment Gmt (Gamma tilde) 
            Gm[li, i+nsteps, j] = G[li, j]
            for k in range(i+1, j+1):
                Gm[li, k+nsteps, j] = Gm[pli, k-1+nsteps, j-1]
                lb = max(i, k-1)
                Gm[li, k+nsteps, j] += d*np.sum(alpha_s(cp,delta*np.arange(lb+nsteps-j+1, nsteps))*(Gm[pli, lb+nsteps:j-1+nsteps, j-1] + 3*G[lb+nsteps-j+1:nsteps, j-1] + 2*G2[lb+nsteps-j+1:nsteps, j-1] + (2-2*Nr)*Gm2[pli, lb+nsteps:j-1+nsteps, j-1] - Nr*Gb[pli, lb+nsteps:j-1+nsteps, j-1] - 2*Nr*Qt[lb+nsteps-j+1:nsteps, j-1]))

            # increment G2
            G2[li, j] = G2[pli, j-1]
            G2[li, j] += 2*d*alpha_s(cp,delta*li)*np.sum(G[li, :j] + 2*G2[li, :j])
            
            # increment Gm2
            Gm2[li, i+nsteps, j] = G2[li, j]
            Gm2[li, i+1+nsteps:j+1+nsteps, j] = Gm2[pli, i+nsteps:j+nsteps, j-1]

            # increment Qt
            Qt[li, j] = Qt[pli, j-1]
            Qt[li, j] -= d*alpha_s(cp,delta*li)*np.sum(Q[li, :j] + 2*G2[li, :j])
            
            # increment I3 (2 Gmt3 - Gm3 - 2 Gm4 + Gm5 + 3 Gm6 - Gm2)(0 4 -4 2 6 -4 -6)
            I3[li, j] = I3[pli, j-1]
            I3[li, j] += d*np.sum(alpha_s(cp,delta*np.arange(pli, nsteps))*(2*Gm3t[pli, i+nsteps:j-1+nsteps, j-1] - Gm3[pli, i+nsteps:j-1+nsteps, j-1] - 2*Gm4[pli, i+nsteps:j-1+nsteps, j-1] + Gm5[pli, i+nsteps:j-1+nsteps, j-1] - Gm2[pli, i+nsteps:j-1+nsteps, j-1]))
            I3[li, j] += 0.5*d*alpha_s(cp,delta*li)*np.sum(4*I3t[li, :j] - 4*I4[li, :j] + 2*I5[li, :j] - 4*G[li, :j] - 6*G2[li, :j])
            
            # increment Gm3 
            Gm3[li, i+nsteps, j] = I3[li, j]
            for k in range(i+1, j+1):
                Gm3[li, k+nsteps, j] = Gm3[pli, k-1+nsteps, j-1]
                lb = max(i, k-1)
                Gm3[li, k+nsteps, j] += d*np.sum(alpha_s(cp,delta*np.arange(lb+nsteps-j+1, nsteps))*(2*Gm3t[pli, lb+nsteps:nsteps+j-1, j-1] - Gm3[pli, lb+nsteps:nsteps+j-1, j-1] - 2*Gm4[pli, lb+nsteps:nsteps+j-1, j-1] + Gm5[pli, lb+nsteps:nsteps+j-1, j-1] - Gm2[pli, lb+nsteps:nsteps+j-1, j-1]))

            # increment I3t (I3 tilde) (Gmt3 - (Nr/4) Gm3 + (Nr/2 - 2) Gm4 + (1-Nr/4)Gm5 + (3- 3Nr/4)Gm6 + (1-Nr/4)Gm2)
            # (-Nr/2 4 Nr-4 2-Nr/2 6-3Nr/2 -4 Nr/2-6)
            I3t[li, j] = I3t[pli, j-1] 
            I3t[li, j] += d*np.sum(alpha_s(cp,delta*np.arange(pli, nsteps))*(Gm3t[pli, i+nsteps:j-1+nsteps, j-1] - Nr*Gm3[pli, i+nsteps:j-1+nsteps, j-1]  + (2*Nr-2)*Gm4[pli, i+nsteps:j-1+nsteps, j-1] + (1-Nr)*Gm5[pli, i+nsteps:j-1+nsteps, j-1] + (Nr-1)*Gm2[pli, i+nsteps:j-1+nsteps, j-1]))
            I3t[li, j] += 0.5*d*alpha_s(cp,delta*li)*np.sum(-2*Nr*I3[li, :j] + 4*I3t[li, :j] + (4*Nr-4)*I4[li, :j] + (2-2*Nr)*I5[li, :j] - 4*G[li, :j] + (2*Nr - 6)*G2[li, :j])
                   
            # increment Gm3t (Gm3 tilde)
            Gm3t[li, i+nsteps, j] = I3t[li, j]
            for k in range(i+1, j+1):
                Gm3t[li, k+nsteps, j] = Gm3t[pli, k-1+nsteps, j-1]
                lb = max(i, k-1)
                Gm3t[li, k+nsteps, j] += d*np.sum(alpha_s(cp,delta*np.arange(lb+nsteps-j+1, nsteps))*(Gm3t[pli, lb+nsteps:nsteps+j-1, j-1] - Nr*Gm3[pli, lb+nsteps:nsteps+j-1, j-1] + (2*Nr-2)*Gm4[pli, lb+nsteps:nsteps+j-1, j-1] + (1-Nr)*Gm5[pli, lb+nsteps:nsteps+j-1, j-1] + (Nr-1)*Gm2[pli, lb+nsteps:nsteps+j-1, j-1]))

            # increment I4(0,1,0,4,2,-2)
            I4[li, j] = I4[pli, j-1]
            I4[li, j] += 0.5*d*alpha_s(cp,delta*li)*np.sum(G2[li, :j] + 4*I4[li, :j] + 2*I5[li, :j])
            
            # increment Gm4
            Gm4[li, i+nsteps, j] = I4[li, j]
            Gm4[li, i+1+nsteps:j+1+nsteps, j] = Gm4[pli, i+nsteps:j+nsteps, j-1]
            
            # increment I5(2,3,-2,2,-1,-3)
            I5[li, j] = I5[pli, j-1]
            I5[li, j] += 0.5*d*alpha_s(cp,delta*li)*np.sum(2*G[li, :j] + 3*G2[li, :j] - 2*I3t[li, :j] + 2*I4[li, :j] - I5[li, :j])
            
            # increment Gm5
            Gm5[li, i+nsteps, j] = I5[li, j]
            Gm5[li, i+1+nsteps:j+1+nsteps, j] = Gm5[pli, i+nsteps:j+nsteps, j-1]
            
        if(np.mod(j, round(nsteps/10)) == 0) and verb: 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")

    print(len(Q))
        
    return [Q, G, G2, Qt, I3, I3t, I4, I5]



In [13]:
# large Nc&Nf (can handle 6/9 = .6 of the memory of large Nc) W/ RC
# eta_maxs = {0.0125:6, 0.016:6, 0.025:12, 0.032:12, 0.0375:18, 0.05:24, 0.0625:30, 0.075:36, 0.08:36, 0.1:42}
eta_maxs = {0.05:24, 0.0625:30, 0.075:36, 0.08:36, 0.1:42}

#     ICs = np.random.randint(-6,6, size=(6))
ICs = "ones"
ICs_tag = "ones"
output_dir = "/Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/"

Nf = 3

for delta, eta_max in eta_maxs.items():
    eta_max = 10
    # delta = 0.05

    if round(eta_max/delta) > 800:
        print("Error: nsteps > 800 and will probably crash your computer")
        break
        
    print("nsteps:", round(eta_max/delta))
    delta_str = str(delta)[2:]
    amp_strs = {0:"Q", 1:"G", 2:"G2", 3:"Qt", 4:"I3", 5:"I3t", 6:"I4", 7:"I5"}
    print('Staring (d, eta_max)=', delta, eta_max)
    
    output = evolve_NcNf_nocutoff(delta, eta_max, IC = ICs, coupling = 'running')

    # save output
    if not os.path.exists(output_dir+"d"+delta_str+"_"+ICs_tag): os.mkdir(output_dir+'d'+delta_str+'_'+ICs_tag)
    for iamp in range(8):
        np.savetxt(output_dir+"d"+delta_str+"_"+ICs_tag+"/d"+delta_str+"_NcNf"+str(Nf)+"_"+ICs_tag+"_"+amp_strs[iamp]+"_nocutoff_rc.dat", output[iamp])
    print("wrote out amplitudes to", output_dir)
    output = 0



nsteps: 200
Staring (d, eta_max)= 0.05 10
Finished initialization
10.0% done (13:04.42)
20.0% done (13:05.03)
30.0% done (13:05.24)
40.0% done (13:05.46)
50.0% done (13:06.07)
60.0% done (13:06.28)
70.0% done (13:06.50)
80.0% done (13:07.12)
90.0% done (13:07.33)
100.0% done (13:07.55)
201
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/
nsteps: 160
Staring (d, eta_max)= 0.0625 10
Finished initialization
10.0% done (13:08.07)
20.0% done (13:08.18)
30.0% done (13:08.29)
40.0% done (13:08.40)
50.0% done (13:08.51)
60.0% done (13:09.02)
70.0% done (13:09.12)
80.0% done (13:09.23)
90.0% done (13:09.34)
100.0% done (13:09.45)
161
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/
nsteps: 133
Staring (d, eta_max)= 0.075 10
Finished initialization
9.8% done (13:09.51)
19.5% done (13:09.57)
29.3% done (13:10.03)
39.1% done (13:10.10)
48.9% done (13:10.16)
58.6% done (13:10.22)
68.4% done (13

In [3]:
# here lambda is a cutoff, also includes Qt corrections from 2406.11647 and It amplitude from 2410.21260
# this one has running coupling

def alpha_s(coupling, s):
    Nc = 3.0
    Nf = 3.0
    beta2 = (11*Nc-2*Nf)/(12*np.pi)
    s0 = 1.0 # log ratio of IR cutoff to \Lambda_{QCD}
    
    if coupling == 'fixed': return 0.25
    elif coupling == 'running': return (np.sqrt(Nc/(2*np.pi))/beta2)*(1/(s0+s))
        

def evolve_NcNf_rc(delta, eta_max, Nf=3.0, ICs="ones", coupling="fixed", s0=1.0):
    Nr = 0.25*(Nf/3.0)
    nsteps = round(eta_max/delta)
    d2 = delta**2
    cp = coupling
    
    G = np.ones((nsteps + 1, nsteps + 1))
    Q = np.ones((nsteps + 1, nsteps + 1))
    G2 = np.ones((nsteps + 1, nsteps + 1))
    Qt = np.ones((nsteps + 1, nsteps + 1))
    GmT = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    GmB = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm2 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

    I3t = np.ones((nsteps + 1, nsteps + 1))
    I3 = np.ones((nsteps + 1, nsteps + 1))
    I4 = np.ones((nsteps + 1, nsteps + 1))
    I5 = np.ones((nsteps + 1, nsteps + 1))
    It = np.ones((nsteps + 1, nsteps + 1))
    Gm3t = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm3 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm4 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm5 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

            
    if type(ICs) != str:  
        for amp, ne_amp, ic in zip([G, Q, G2, I3t, I3, I4, I5], [GmT, GmB, Gm2, Gm3t, Gm3, Gm4, Gm5], ICs):
            amp *= ic
            ne_amp *= ic
     
    print("amplitudes initialized to", ICs)
    print("Nf=", Nf)
        
    for j in range(1, nsteps + 1):
        for i in range(j):
            
            Q[i, j] = Q[i, j-1]
            Q[i, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(Q[i:j-1, j-1] + 2*G2[i:j-1, j-1]))
            Q[i, j] += 0.5*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(Q[:i, j-i:j] + 2*G2[:i, j-i:j]))
            Q[i, j] += d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(2*G[i:j-1,j-1] + Q[i:j-1,j-1] + 2*G2[i:j-1,j-1] + 2*GmT[i, i:j-1, j-1] - GmB[i, i:j-1, j-1] + 2*Gm2[i, i:j-1, j-1]))
            
            GmB[i, i, j] = Q[i, j]
            for k in range(i+1, j+1):
                GmB[i, k, j] = GmB[i, k-1, j-1]
                GmB[i, k, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(k-1, j-1))*(Q[k-1:j-1, j-1] + 2*G2[k-1:j-1, j-1]))
                ii = max(i, k-1)
                GmB[i, k, j] += d2*np.sum(alpha_s(cp,delta*np.arange(ii, j-1))*(2*GmT[i, ii:j-1, j-1] - GmB[i, ii:j-1, j-1] + 2*Gm2[i, ii:j-1, j-1] + 2*G[ii:j-1, j-1] + Q[ii:j-1, j-1] + 2*G2[ii:j-1, j-1]))
            
            G[i, j] = G[i, j-1]
            G[i, j] -= Nr*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(Q[:i, j-i:i] + 2*G2[:i, j-i:i]))
            G[i, j] += d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(3*G[i:j-1, j-1] + 2*G2[i:j-1, j-1] - 2*Nr*Qt[i:j-1, j-1]))
            G[i, j] += d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(GmT[i, i:j-1, j-1] + (2-2*Nr)*Gm2[i, i:j-1, j-1] - Nr*GmB[i, i:j-1, j-1]))
            
            GmT[i, i, j] = G[i, j]
            for k in range(i+1, j+1):
                GmT[i, k, j] = GmT[i, k-1, j-1]
                ii = max(i, k-1)
                GmT[i, k, j] += d2*np.sum(alpha_s(cp,delta*np.arange(ii, j-1))*(GmT[i, ii:j-1, j-1] + (2-2*Nr)*Gm2[i, ii:j-1, j-1] - Nr*GmB[i, ii:j-1, j-1] + 3*G[ii:j-1, j-1] + 2*G2[ii:j-1, j-1] - 2*Nr*Qt[ii:j-1, j-1]))
            
            G2[i, j] = G2[i, j-1]
            G2[i, j] += 2*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(G[:i, j-i:j] + 2*G2[:i, j-i:j]))

            Gm2[i, i, j] = G2[i, j]
            Gm2[i, i+1:j+1, j] = Gm2[i, i:j, j-1]

            Qt[i, j] = Qt[i, j-1]
            Qt[i, j] -= d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(Q[:i, j-i:j] + 2*G2[:i, j-i:j]))
              
            I3[i, j] = I3[i, j-1]
            I3[i, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(4*Gm3t[i, i:j-1, j-1] - 2*Gm3[i, i:j-1, j-1] - 4*Gm4[i, i:j-1, j-1] + 2*Gm5[i, i:j-1, j-1] - 2*Gm2[i, i:j-1, j-1]))
            I3[i, j] += 0.5*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(4*I3t[:i, j-i:j] - 4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 4*G[:i, j-i:j] - 6*G2[:i, j-i:j]))
            
            Gm3[i, i, j] = I3[i, j]
            for k in range(i+1, j+1):
                Gm3[i, k, j] = Gm3[i, k-1, j-1]
                ii = max(i, k-1)
                Gm3[i, k, j] += d2*np.sum(alpha_s(cp,delta*np.arange(ii, j-1))*(2*Gm3t[i, ii:j-1, j-1] - Gm3[i, ii:j-1, j-1] - 2*Gm4[i, ii:j-1, j-1] + Gm5[i, ii:j-1, j-1] - Gm2[i, ii:j-1, j-1]))
            
            I3t[i, j] = I3t[i, j-1]
            I3t[i, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(2*Gm3t[i, i:j-1, j-1] - 2*Nr*Gm3[i, i:j-1, j-1] + (4*Nr - 4)*Gm4[i, i:j-1, j-1] + (2 - 2*Nr)*Gm5[i, i:j-1, j-1] + (2*Nr - 2)*Gm2[i, i:j-1, j-1]))
            I3t[i, j] += 0.5*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(-2*Nr*I3[:i, j-i:j] + 4*I3t[:i, j-i:j] + (4*Nr-4)*I4[:i, j-i:j] + (2 - 2*Nr)*I5[:i, j-i:j] - 4*G[:i, j-i:j] + (2*Nr - 6)*G2[:i, j-i:j]))
            
            Gm3t[i, i, j] = I3t[i, j]
            for k in range(i+1, j+1):
                Gm3t[i, k, j] = Gm3t[i, k-1, j-1]
                Gm3t[i, k, j] += d2*np.sum(delta*alpha_s(cp,np.arange(ii, j-1))*(Gm3t[i, ii:j-1, j-1] - Nr*Gm3[i, ii:j-1, j-1] + (2*Nr - 2)*Gm4[i, ii:j-1, j-1] + (1 - Nr)*Gm5[i, ii:j-1, j-1] + (Nr - 1)*Gm2[i, ii:j-1, j-1]))

            I4[i, j] = I4[i, j-1]
            I4[i, j] += 0.5*d2*np.trace(delta*alpha_s(cp,np.arange(i))[:, np.newaxis]*(4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] + G2[:i, j-i:j]))
            
            Gm4[i, i, j] = I4[i, j]
            Gm4[i, i+1:j+1, j] = Gm4[i, i:j, j-1]
            
            I5[i, j] = I5[i, j-1]
            I5[i, j] += 0.5*d2*np.trace(delta*alpha_s(cp,np.arange(i))[:, np.newaxis]*(-2*I3t[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j] + 2*G[:i, j-i:j] + 3*G2[:i, j-i:j]))
            
            Gm5[i, i, j] = I5[i, j]
            Gm5[i, i+1:j+1, j] = Gm5[i, i:j, j-1]

            It[i, j] = It[i, j-1]
            It[i, j] -= 0.5*d2*np.trace(delta*alpha_s(cp,np.arange(i))[:, np.newaxis]*(Q[:i, j-i:j] + 3*G2[:i, j-i:j] - I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j]))
                
        if(np.mod(j, round(nsteps/10)) == 0): 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")

    return [Q, G, G2, Qt, I3, I3t, I4, I5, It]
    

In [19]:
# large Nc&Nf (can handle 6/9 = .6 of the memory of large Nc) W/ RC
eta_maxs = {0.0125:6, 0.016:6, 0.025:12, 0.032:12, 0.0375:18, 0.05:24, 0.0625:30, 0.075:36, 0.08:36, 0.1:42}

#     ICs = np.random.randint(-6,6, size=(6))
ICs = "ones"
ICs_tag = "ones"
output_dir = "/Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/"

Nf = 3

# for delta, eta_max in eta_maxs.items():
for delta in list(eta_maxs.keys())[::-1]:
    
    eta_max = 10
    # delta = 0.05

    if round(eta_max/delta) > 800:
        print("Error: nsteps > 800 and will probably crash your computer")
        break
        
    print("nsteps:", round(eta_max/delta)) 
    delta_str = str(delta)[2:]
    amp_strs = {0:"Q", 1:"G", 2:"G2", 3:"Qt", 4:"I3", 5:"I3t", 6:"I4", 7:"I5", 8:"It"}
    print('Staring (d, eta_max)=', delta, eta_max)
    
    output = evolve_NcNf_rc(delta, eta_max, ICs = ICs, coupling='running')

    # save output
    if not os.path.exists(output_dir+"d"+delta_str+"_"+ICs_tag): os.mkdir(output_dir+"d"+delta_str+"_"+ICs_tag)
    for iamp in range(9):
        np.savetxt(output_dir+"d"+delta_str+"_"+ICs_tag+"/d"+delta_str+"_NcNf"+str(Nf)+"_"+ICs_tag+"_"+amp_strs[iamp]+"_rc.dat", output[iamp])
    print("wrote out amplitudes to", output_dir)
    output = 0



nsteps: 100
Staring (d, eta_max)= 0.1 10
amplitudes initialized to ones
Nf= 3.0
10.0% done (09:24.53)
20.0% done (09:24.54)
30.0% done (09:24.54)
40.0% done (09:24.54)
50.0% done (09:24.55)
60.0% done (09:24.56)
70.0% done (09:24.57)
80.0% done (09:24.58)
90.0% done (09:25.00)
100.0% done (09:25.03)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/
nsteps: 125
Staring (d, eta_max)= 0.08 10
amplitudes initialized to ones
Nf= 3.0
9.6% done (09:25.03)
19.2% done (09:25.03)
28.8% done (09:25.03)
38.4% done (09:25.04)
48.0% done (09:25.05)
57.6% done (09:25.07)
67.2% done (09:25.09)
76.8% done (09:25.11)
86.4% done (09:25.15)
96.0% done (09:25.19)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/
nsteps: 133
Staring (d, eta_max)= 0.075 10
amplitudes initialized to ones
Nf= 3.0
9.8% done (09:25.22)
19.5% done (09:25.22)
29.3% done (09:25.22)
39.1% done (09:25.23)
48.9% done (09:25.24)
58.6